In [5]:
# Import pkgs
import numpy as np 
import pandas as pd
from datetime import datetime

# Working Dirs
RAW_DATA_PATH = r"D:\research\IP_NETWORK_ANALYSIS\IP_NETWORK_ANALYSIS\data\raw\RawIPDataset.csv"

# Notebook settings
pd.set_option('display.max_columns', None)
chunksize = 1_000_000

# Custom date parser function
def custom_date_parser(date_string):
    return datetime.strptime(date_string, '%d/%m/%Y%H:%M:%S')

In [65]:
%timeit
# Read in data
index_df = pd.read_csv(RAW_DATA_PATH,
                 usecols=["Source.IP","Source.Port","Destination.IP",
                          "Destination.Port","Protocol","Timestamp",
                          "ProtocolName"],
                 parse_dates=["Timestamp"],
                 # date_parser=custom_date_parser,
                 date_format = '%d/%m/%Y%H:%M:%S',
                 # nrows = chunksize,
                 low_memory=False)

# Split date and time from Time Stamp
index_df['date'] = index_df['Timestamp'].dt.date
index_df['time'] = index_df['Timestamp'].dt.time

# Seperate Network address from host
index_df['rfind_idx'] = index_df['Source.IP'].str.rfind('.')
index_df["network.addr"] = index_df.apply(lambda df: df["Source.IP"][0: df["rfind_idx"]], axis=1)
index_df["network.host"] = index_df.apply(lambda df: df["Source.IP"][df["rfind_idx"]+1:], axis=1)
index_df.drop(columns=["rfind_idx"], inplace=True)

# Sort the columns
sort_cols_list = ['Source.IP', 'Source.Port', 'Destination.IP', 'Destination.Port',
                  'network.addr', 'network.host','Protocol','ProtocolName',
                  'Timestamp','date','time']
index_df = index_df[sort_cols_list]
 
index_df.sample(10)

,Source.IP,Source.Port,Destination.IP,Destination.Port,network.addr,network.host,Protocol,ProtocolName,Timestamp,date,time
3347671,10.200.7.195,40228,199.16.156.21,443,10.200.7,195,6,TWITTER,2017-05-15 04:40:19,2017-05-15,04:40:19
1427718,10.200.7.217,40460,172.217.29.42,443,10.200.7,217,6,GOOGLE,2017-04-27 06:08:57,2017-04-27,06:08:57
3267224,10.200.7.8,3128,192.168.32.106,52997,10.200.7,8,6,HTTP_PROXY,2017-05-15 11:15:20,2017-05-15,11:15:20
858507,10.200.7.194,47964,179.1.4.217,443,10.200.7,194,6,SSL,2017-04-27 10:40:45,2017-04-27,10:40:45
1921233,10.200.7.199,38825,172.217.29.74,443,10.200.7,199,6,GOOGLE,2017-05-09 09:10:02,2017-05-09,09:10:02
2948328,192.168.10.114,63522,10.200.7.7,3128,192.168.10,114,6,YOUTUBE,2017-05-11 03:32:09,2017-05-11,03:32:09
2316724,10.200.7.199,37295,31.13.73.7,443,10.200.7,199,6,FACEBOOK,2017-05-11 09:27:50,2017-05-11,09:27:50
1320236,10.200.7.9,3128,192.168.130.9,50334,10.200.7,9,6,HTTP,2017-04-27 04:55:44,2017-04-27,04:55:44
1060534,10.200.7.9,3128,172.19.1.47,55457,10.200.7,9,6,HTTP_PROXY,2017-04-27 11:30:02,2017-04-27,11:30:02
3102379,10.200.7.5,3128,192.168.41.14,61506,10.200.7,5,6,HTTP,2017-05-15 09:54:35,2017-05-15,09:54:35


In [71]:
index_df[(index_df["network.addr"]!="10.200.7") & (index_df["ProtocolName"]=="TWITTER")]["network.addr"].value_counts()

network.addr
192.168.72     1220
192.168.10     1019
192.168.130     782
192.168.42      708
192.168.32      652
192.168.180     513
192.168.60      424
192.168.81      403
192.168.90      398
192.168.41      269
192.168.220     252
10.230.1        249
192.168.52      225
192.168.151     162
192.168.40      154
192.168.131     148
192.168.110     136
192.168.29      127
192.168.112     119
192.168.142     102
192.168.205      97
104.244.46       87
192.168.202      83
199.16.156       44
104.244.43       44
104.244.42       42
172.19.1         41
199.96.57        38
192.168.190      36
192.168.51       33
192.168.50       25
172.21.1         20
192.168.150      17
172.18.1         16
192.168.102      14
192.229.163      10
72.21.91         10
192.168.31       10
199.16.157        9
93.184.216        8
192.168.171       6
172.22.1          5
192.16.58         2
192.16.59         2
192.168.128       1
192.168.122       1
172.17.1          1
Name: count, dtype: int64

In [75]:
# index_df[(index_df["ProtocolName"]=="TWITTER") & (index_df["network.addr"]=="192.168.72")]
index_df[index_df["network.addr"]=="10.230.1"].ProtocolName.value_counts()

ProtocolName
HTTP_PROXY        25044
GOOGLE            22342
HTTP_CONNECT      12601
YOUTUBE            5509
MICROSOFT          2086
GMAIL              1424
HTTP               1319
WINDOWS_UPDATE     1150
SKYPE               832
FACEBOOK            507
AMAZON              412
MSN                 317
TWITTER             249
WHATSAPP            200
OFFICE_365          122
SPOTIFY              65
YAHOO                59
GOOGLE_MAPS          53
SSL                  38
DROPBOX              28
LOTUS_NOTES           4
INSTAGRAM             3
FTP_DATA              2
EASYTAXI              2
NETFLIX               2
Name: count, dtype: int64

In [23]:
index_df["Flow.ID"] = index_df["Source.Port"].astype(str)+"@"+index_df["Source.IP"]+\
    " --> "+index_df["Destination.Port"].astype(str)+"@"+index_df["Destination.IP"]

index_df["Flow.ID"].sample(10)


2033111    38086@10.200.7.195 --> 8081@200.25.59.96
1326068      51333@10.230.1.119 --> 3128@10.200.7.8
3173091     58172@192.168.90.98 --> 3128@10.200.7.9
3274324      56071@10.230.1.213 --> 3128@10.200.7.7
3123152     3128@10.200.7.9 --> 63017@192.168.90.98
938590     4965@192.168.130.245 --> 3128@10.200.7.8
545381      64101@192.168.10.37 --> 3128@10.200.7.9
2213452    3128@10.200.7.4 --> 55294@192.168.52.202
519812      3128@10.200.7.6 --> 62782@192.168.41.10
1406458       33689@10.200.7.196 --> 80@179.1.4.210
Name: Flow.ID, dtype: object

In [26]:
index_df["RevFlow.ID"] = index_df["Destination.Port"].astype(str)+"@"+index_df["Destination.IP"]+\
    " --> "+index_df["Source.Port"].astype(str)+"@"+index_df["Source.IP"]

index_df

,Source.IP,Source.Port,Destination.IP,Destination.Port,Protocol,Timestamp,ProtocolName,date,time,Flow.ID,RevFlow.ID
0,172.19.1.46,52422,10.200.7.7,3128,6,2017-04-26 11:11:17,HTTP_PROXY,2017-04-26,11:11:17,52422@172.19.1.46 --> 3128@10.200.7.7,3128@10.200.7.7 --> 52422@172.19.1.46
1,10.200.7.7,3128,172.19.1.46,52422,6,2017-04-26 11:11:17,HTTP_PROXY,2017-04-26,11:11:17,3128@10.200.7.7 --> 52422@172.19.1.46,52422@172.19.1.46 --> 3128@10.200.7.7
2,50.31.185.39,80,10.200.7.217,38848,6,2017-04-26 11:11:17,HTTP,2017-04-26,11:11:17,80@50.31.185.39 --> 38848@10.200.7.217,38848@10.200.7.217 --> 80@50.31.185.39
3,50.31.185.39,80,10.200.7.217,38848,6,2017-04-26 11:11:17,HTTP,2017-04-26,11:11:17,80@50.31.185.39 --> 38848@10.200.7.217,38848@10.200.7.217 --> 80@50.31.185.39
4,192.168.72.43,55961,10.200.7.7,3128,6,2017-04-26 11:11:17,HTTP_PROXY,2017-04-26,11:11:17,55961@192.168.72.43 --> 3128@10.200.7.7,3128@10.200.7.7 --> 55961@192.168.72.43
...,...,...,...,...,...,...,...,...,...,...,...
3577291,98.138.79.73,443,10.200.7.199,42135,6,2017-05-15 05:43:40,SSL,2017-05-15,05:43:40,443@98.138.79.73 --> 42135@10.200.7.199,42135@10.200.7.199 --> 443@98.138.79.73
3577292,98.138.79.73,443,10.200.7.217,51546,6,2017-05-15 05:46:10,SSL,2017-05-15,05:46:10,443@98.138.79.73 --> 51546@10.200.7.217,51546@10.200.7.217 --> 443@98.138.79.73
3577293,98.138.79.73,443,10.200.7.218,44366,6,2017-05-15 05:45:39,SSL,2017-05-15,05:45:39,443@98.138.79.73 --> 44366@10.200.7.218,44366@10.200.7.218 --> 443@98.138.79.73
3577294,98.138.79.73,443,10.200.7.195,52341,6,2017-05-15 05:45:59,SSL,2017-05-15,05:45:59,443@98.138.79.73 --> 52341@10.200.7.195,52341@10.200.7.195 --> 443@98.138.79.73


In [36]:
index_df.loc[index_df["Flow.ID"]=="52422@172.19.1.46 --> 3128@10.200.7.7"]

,Source.IP,Source.Port,Destination.IP,Destination.Port,Protocol,Timestamp,ProtocolName,date,time,Flow.ID,RevFlow.ID
0,172.19.1.46,52422,10.200.7.7,3128,6,2017-04-26 11:11:17,HTTP_PROXY,2017-04-26,11:11:17,52422@172.19.1.46 --> 3128@10.200.7.7,3128@10.200.7.7 --> 52422@172.19.1.46


In [37]:
index_df.loc[index_df["Flow.ID"]=="3128@10.200.7.7 --> 52422@172.19.1.46"]

,Source.IP,Source.Port,Destination.IP,Destination.Port,Protocol,Timestamp,ProtocolName,date,time,Flow.ID,RevFlow.ID
1,10.200.7.7,3128,172.19.1.46,52422,6,2017-04-26 11:11:17,HTTP_PROXY,2017-04-26,11:11:17,3128@10.200.7.7 --> 52422@172.19.1.46,52422@172.19.1.46 --> 3128@10.200.7.7
99574,10.200.7.7,3128,172.19.1.46,52422,6,2017-04-26 11:11:17,HTTP_PROXY,2017-04-26,11:11:17,3128@10.200.7.7 --> 52422@172.19.1.46,52422@172.19.1.46 --> 3128@10.200.7.7


In [31]:
index_df.loc[index_df["Flow.ID"]=="3128@10.200.7.7 --> 52422@172.19.1.46"]

,Source.IP,Source.Port,Destination.IP,Destination.Port,Protocol,Timestamp,ProtocolName,date,time,Flow.ID,RevFlow.ID
1,10.200.7.7,3128,172.19.1.46,52422,6,2017-04-26 11:11:17,HTTP_PROXY,2017-04-26,11:11:17,3128@10.200.7.7 --> 52422@172.19.1.46,52422@172.19.1.46 --> 3128@10.200.7.7
99574,10.200.7.7,3128,172.19.1.46,52422,6,2017-04-26 11:11:17,HTTP_PROXY,2017-04-26,11:11:17,3128@10.200.7.7 --> 52422@172.19.1.46,52422@172.19.1.46 --> 3128@10.200.7.7


In [32]:
index_df[index_df["ProtocolName"]=="YOUTUBE"]

,Source.IP,Source.Port,Destination.IP,Destination.Port,Protocol,Timestamp,ProtocolName,date,time,Flow.ID,RevFlow.ID
36,192.168.72.43,55980,10.200.7.7,3128,6,2017-04-26 11:11:17,YOUTUBE,2017-04-26,11:11:17,55980@192.168.72.43 --> 3128@10.200.7.7,3128@10.200.7.7 --> 55980@192.168.72.43
37,10.200.7.217,37557,179.1.4.251,443,6,2017-04-26 11:11:17,YOUTUBE,2017-04-26,11:11:17,37557@10.200.7.217 --> 443@179.1.4.251,443@179.1.4.251 --> 37557@10.200.7.217
38,10.200.7.217,37557,179.1.4.251,443,6,2017-04-26 11:11:17,YOUTUBE,2017-04-26,11:11:17,37557@10.200.7.217 --> 443@179.1.4.251,443@179.1.4.251 --> 37557@10.200.7.217
83,192.168.42.31,52680,10.200.7.5,3128,6,2017-04-26 11:11:18,YOUTUBE,2017-04-26,11:11:18,52680@192.168.42.31 --> 3128@10.200.7.5,3128@10.200.7.5 --> 52680@192.168.42.31
84,10.200.7.195,44858,216.58.202.225,443,6,2017-04-26 11:11:18,YOUTUBE,2017-04-26,11:11:18,44858@10.200.7.195 --> 443@216.58.202.225,443@216.58.202.225 --> 44858@10.200.7.195
...,...,...,...,...,...,...,...,...,...,...,...
3575798,192.168.90.104,1679,10.200.7.7,3128,6,2017-05-15 05:40:28,YOUTUBE,2017-05-15,05:40:28,1679@192.168.90.104 --> 3128@10.200.7.7,3128@10.200.7.7 --> 1679@192.168.90.104
3575799,192.168.90.104,1679,10.200.7.7,3128,6,2017-05-15 05:45:01,YOUTUBE,2017-05-15,05:45:01,1679@192.168.90.104 --> 3128@10.200.7.7,3128@10.200.7.7 --> 1679@192.168.90.104
3575801,192.168.90.104,1679,10.200.7.7,3128,6,2017-05-15 05:45:01,YOUTUBE,2017-05-15,05:45:01,1679@192.168.90.104 --> 3128@10.200.7.7,3128@10.200.7.7 --> 1679@192.168.90.104
3575803,192.168.90.104,1653,10.200.7.7,3128,6,2017-05-15 05:44:43,YOUTUBE,2017-05-15,05:44:43,1653@192.168.90.104 --> 3128@10.200.7.7,3128@10.200.7.7 --> 1653@192.168.90.104
